In [1]:
import cv2 as cv
from skimage import io, measure
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from imgbasics import contour_properties, closest_contour, imcrop, contour_coords
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
img_base = io.imread('data/example.png')

# Image cropping

## Interactive mode, crop defined with clicks

In [3]:
img, cropzone = imcrop(img_base)
print(cropzone)
img.shape

(219, 223, 139, 71)


(71, 139)

## Interactive mode, crop defined with a draggable rectangle

In [4]:
img, cropzone = imcrop(img_base, draggable=True)
print(cropzone)
img.shape

Cursor disconnected (max number of clicks, or stop button pressed).
Rectangle position recorded. Rectangle deleted.
(220, 225, 133, 71)


(71, 133)

## Non-interactive mode

In [5]:
cropzone = 200, 215, 160, 100
img = imcrop(img_base, cropzone)
plt.imshow(img, cmap='gray')

## Contour detection using scikit-image or openCV

In [6]:
img = imcrop(img_base, (200, 215, 160, 100))
glevel =   220     # gray level at which contours are plotted
position = 100, 50  # we will only keep the contour closest to this position

# Contours generated by scikit-image
contours_sk = measure.find_contours(img, glevel)

# Contours generated by opencv
ret, thresh = cv.threshold(img, glevel,255,0)
contours_cv, hierarchy = cv.findContours(thresh, 1, 2)

# Use closecont to only keep the contour that has its edge closest to the preset position
contour_sk = closest_contour(contours_sk, position, edge=True, source='scikit')
contour_cv = closest_contour(contours_cv, position, edge=True, source='opencv')

# Convert both to usable x, y coordinates
x_sk, y_sk = contour_coords(contour_sk, source='scikit')
x_cv, y_cv = contour_coords(contour_cv, source='opencv')

In [7]:
fig, ax = plt.subplots()

ax.imshow(img, cmap='gray')

ax.plot(x_sk, y_sk, '-r', linewidth=2)
ax.plot(x_cv, y_cv, '.-g', linewidth=1)

## Direction of contours (clockwise or counter)

In [8]:
fig, ax = plt.subplots()
xs, ys = x_sk, y_sk  # here, change which contour you would like to plot (colors start from blue and end in yellow)
m = cm.get_cmap('viridis', len(xs))
ax.imshow(img, cmap='gray')
for i, (x, y) in enumerate(zip(xs, ys)):
    ax.plot(x, y, 'o', c=m.colors[i])

As one can see by exploring with the code above, the results vary depending on the pakage used for the analysis and the situation. As a summary (A refers to the signed area measured by contprops(), see below). In scikit-image it is possible to get opposite directions to those below by using the `positive_orientation='high'` option.

particle type           | scikit-image            | opencv
:---:                   | :---:                   | :---:
dark (light background) | counter-clockwise (A>0) | clockwise (A<0)
light (dark background) | clockwise (A<0)         | counter-clockwise (A>0) 

## Contour properties according to imgbasics

The tuples returned below correspond to `x, y, w, h` where `x, y` is the centroid position, `p` is the perimeter, `a` the area.

In [9]:
contour_properties(x_sk, y_sk)

{'centroid': (102.55280932436305, 56.66187062458395),
 'perimeter': 43.67285746941818,
 'area': -148.1689032693281}

In [10]:
contour_properties(x_cv, y_cv)

{'centroid': (102.5, 56.765151515151516),
 'perimeter': 43.79898987322332,
 'area': 132.0}

## Contour properties according to openCV

Note that opencv, contrary to scikit and imgbasics, can only deal with integer positions in the contour data.

In [11]:
def props_cv(contour):
    """Contour properties with openCV contour as input"""
    M = cv.moments(contour)
    area = M['m00']
    xc = M['m10'] / M['m00']
    yc = M['m01'] / M['m00']
    perimeter = cv.arcLength(contour, True)
    return xc, yc, perimeter, area

def contprops_cv(x, y):
    """Contour properties with explicit x, y as input"""
    contour = np.zeros((len(x), 1, 2), dtype=int)  # openCV only works with int positions
    contour[:, 0, 0] = x
    contour[:, 0, 1] = y
    return props_cv(contour)

In [12]:
contprops_cv(x_sk, y_sk)

(102.11771177117711, 56.385038503850375, 50.72792184352875, 151.5)

In [13]:
contprops_cv(x_cv, y_cv)

(102.5, 56.765151515151516, 43.79898953437805, 132.0)